# Feature Building

## Notebook Setup

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

In [ ]:
pd.set_option('mode.chained_assignment', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Ignore Warnings
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

## Import Data

In [ ]:
gcb = pd.read_parquet("/work/data/Global_Coral_Bleaching_DB/gcb_v3.parquet")
country_code_df = pd.read_csv("/work/data/References/country_name_codes.csv")
meow = gpd.read_file("/work/data/MEOW/meow_ecos.shp")
wdi = pd.read_csv("/work/data/World_Bank/wdi_1980_to_2020.csv")

## Feature Building

### Create Percent_Bleached_Value

In [ ]:
def pct_bleached_calc(row):
    '''
    Function to determine Percent_Bleached_Value
    param row: Pandas dataframe row object
    return: Percent Bleached
    '''

    # Set the default value to None
    bleach_val = None

    # Create a dict for the Donner severity codes
    # These values are currently hard coded to the mean of the range
    #   but could be imputed more programmatically using a distribution
    severity_pcts = {-1:0, 0:0, 1:5, 2:30, 3:75}

    # Create a dict for the Safaie prevalence score
    # These values are currently hard coded to the mean of the range
    #   but could be imputed more programmatically using a distribution
    prevalence_pcts = {0:0, 1:5, 2:17.5, 3:37.5, 4:75}

    # Reef Check uses sectors ... get a mean score
    quad_avg = row[['S1','S2','S3','S4']].mean()

    # Donner & Safaie use a severity code to indicate a general amount of bleaching
    severity_est = severity_pcts.get(row['Severity_ID'], np.nan)
    prevalence_est = prevalence_pcts.get(row['Bleaching_Prevalence_Score_ID'])

    # Grab the Percent Bleached value
    pct_bleached = row['Percent_Bleached']

    # In order of preference for use:
    # Percent_Bleached, the Quad Average, then Severity and Prevalence guesstimates
    if not pd.isna(pct_bleached):
        bleach_val = pct_bleached
    elif not pd.isna(quad_avg):
        bleach_val = quad_avg
    elif not pd.isna(severity_est):
        bleach_val = severity_est
    elif not pd.isna(prevalence_est):
        bleach_val = prevalence_est
    else:
        bleach_val = np.nan
    
    #print(bleach_val, pct_bleached, quad_avg, severity_est, prevalance_est)

    # If the value is still nan (ie, no bleaching data available), return the nan
    # Else look at the value and if it's smaller than 1, multiply by 100 and return
    if pd.isna(bleach_val):
        return bleach_val
    elif bleach_val < 1:
        bleach_val = bleach_val * 100
        if bleach_val >= 1:
            return bleach_val
        else:
            return bleach_val * 100
    else:
        return bleach_val

In [ ]:
# Create the Percent_Bleached_Value
gcb['Percent_Bleached_Value'] = gcb.apply(pct_bleached_calc, axis=1)

# Indicated which rows had the Percent_Bleached_Value imputed
gcb['Bleached_Value_Imputed'] = (~gcb["Percent_Bleached_Value"].isna()) & \
                                   (gcb['Percent_Bleached'].isna()) & \
                                   (gcb['S1'].isna()) & (gcb['S2'].isna()) & \
                                   (gcb['S3'].isna()) & (gcb['S4'].isna())

### Validate Percent_Bleached_Value

In [ ]:
# Show records where Percent_Bleached_Value between 0 and 1
# gcb[(gcb.Percent_Bleached_Value > 0) & (gcb.Percent_Bleached_Value < 1)]

# Count number of records where Percent_Bleached_Value == 0
len(gcb[gcb.Percent_Bleached_Value == 0])

In [ ]:
# How many of those 0% Percent_Bleached_Values were before the year 2010?
print("Total 0% Records Prior to 2010: ", len(gcb[(gcb.Percent_Bleached_Value == 0) & (gcb.Date_Year < 2010)]))
print("Percentage of 0% Records Prior to 2010: ", len(gcb[(gcb.Percent_Bleached_Value == 0) & (gcb.Date_Year < 2010)])/len(gcb.Date_Year < 2010)*100)

In [ ]:
# How many of those 0% Percent_Bleached_Values were after the year 2010?
print("Total 0% Records After 2010: ", len(gcb[(gcb.Percent_Bleached_Value == 0) & (gcb.Date_Year >= 2010)]))
print("Percentage of 0% Records After 2010: ", len(gcb[(gcb.Percent_Bleached_Value == 0) & (gcb.Date_Year >= 2010)])/len(gcb.Date_Year >= 2010)*100)

### Dates

In [ ]:
# Create a composite date column from the day/month/year columns
gcb['Date'] = pd.to_datetime({
    'year': gcb['Date_Year'],
    'month': gcb['Date_Month'],
    'day': gcb['Date_Day']
})

In [ ]:
# Create short month names
month_short_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
gcb['Month_Name'] = gcb['Date_Month'].map(month_short_names)

In [ ]:
# Create a datetime object at a Monthly grain
gcb['Month_Year'] = gcb['Date'].dt.to_period('M')

### Countries

In [ ]:
# Add a country code variable for easy joining with other datasets
gcb = pd.merge(
    gcb, country_code_df[['Country_Name','Country_Code']], 
    how="left", left_on="Country_Name", right_on="Country_Name")

### Nominal Features from Categoricals

In [ ]:
# Create Nominal Features from object dtypes
gcb['Exposure_Cat'] = gcb['Exposure'].astype('category').cat.codes
gcb['Country_Name_Cat'] = gcb['Country_Name'].astype('category').cat.codes

### Add MEOW Province

In [ ]:
# Setup the dataframes as geopandas assets
# Create a point geometry for the gcb dataset
gdf_primary = gpd.GeoDataFrame(gcb, geometry=gcb.apply(lambda row: Point(row['Longitude_Degrees'], row['Latitude_Degrees']), axis=1))
gdf_secondary = gpd.GeoDataFrame(meow, geometry=meow['geometry'])

# Perform a spatial join using a 'within' operation and drop the right join index
joined_gdf = gpd.sjoin(gdf_primary, gdf_secondary[['PROVINCE', 'geometry']], how='left', op='within').drop(columns=['index_right','geometry'])

# Convert the resulting dataframe back to a standard pandas df
# No need to drop the geometry column at this time
gcb = pd.DataFrame(joined_gdf)

In [ ]:
# For some reason, there are 2 Southeast Polynesia datapoints 
#   that don't quite fall within the polygon shapes.  Impute them.
gcb.PROVINCE.fillna('Southeast Polynesia', inplace=True)

### Add World Development Indicators

In [ ]:
# Filter the WDI dataset on the following WDI Codes
# Fishing, Timber, Forest, Marine, Agriculture, Fertilizer, Tourism, and Pop Totals
filter_codes = [
    'AG.LND.IRIG.AG.ZS','AG.LND.AGRI.ZS','AG.LND.AGRI.K2',
    'NV.AGR.TOTL.ZS','NV.AGR.TOTL.KD.ZG','NV.AGR.TOTL.CD',
    'ER.FSH.AQUA.MT','ER.FSH.CAPT.MT','AG.CON.FERT.ZS',
    'EN.FSH.THRD.NO','AG.LND.FRST.ZS','AG.LND.FRST.K2',
    'ST.INT.ARVL','ER.MRN.PTMR.ZS','EN.POP.DNST',
    'SP.POP.TOTL','ER.PTD.TOTL.ZS','ER.FSH.PROD.MT'
    ]
wdi_filtered = wdi[wdi['Series_Code'].isin(filter_codes)]

In [ ]:
# Impute missing WDI Values

# Define year columns
columns_years = [str(year) for year in range(1980, 2021)]

# Impute AG.LND.IRIG.AG.ZS where totally missing
map_fert_pct = {
               'ATG': 7.69, 'BHS': 8.33, 'BRB': 39.00, 'BLZ': 3.48, 'BMU': 0, 
               'KHM': 9.17, 'CYM': 7.69, 'CHN': 51.48, 'COM': 0.13, 'CRI': 18.14,
               'CUB': 15.68, 'DJI': 100, 'DMA': 0.87, 'EGY': 100, 'ERI': 5.49,
               'FJI': 1.59, 'PYF': 3.88, 'IDN': 41.54, 'KEN': 2.34, 'KIR': 0,
               'KWT': 67.6, 'MDV': 0, 'MHL': 0, 'FSM': 0, 'NCL': 0, 'NIC': 10.89,
               'MNP': 0, 'PLW': 0, 'PNG': 0, 'WSM': 0, 'STP': 9.7, 'SGP': 0,
               'SLB': 0.04, 'LKA': 29.23, 'KNA': 0.49, 'LCA': 32.26,
               'VCT': 5.98, 'TZA': 2.32, 'THA': 33.76, 'TON': 0, 'TCA': 0, 
               'TUV': 0, 'VUT': 0, 'VNM': 48.67, 'YEM': 41.8
               }

wdi_filtered['temp_fert_pct'] = wdi_filtered['Country_Code'].map(map_fert_pct)
for col in columns_years:
    wdi_filtered[col] = wdi_filtered[col].fillna(wdi_filtered['temp_fert_pct'])
wdi_filtered.drop('temp_fert_pct', axis=1, inplace=True)


# Impute AG.CON.FERT.ZS where totally missing
map_fert_kgha = {'TCA': 0, 'TUV':0}

wdi_filtered['map_fert_kgha'] = wdi_filtered['Country_Code'].map(map_fert_kgha)
for col in columns_years:
    wdi_filtered[col] = wdi_filtered[col].fillna(wdi_filtered['map_fert_kgha'])
wdi_filtered.drop('map_fert_kgha', axis=1, inplace=True)



# Impute All codes where partially missing
wdi_filtered['mean_1980_2000'] = wdi_filtered[columns_years].mean(axis=1)

for column in columns_years:
    wdi_filtered[column] = wdi_filtered.apply(lambda row: row['mean_1980_2000'] if pd.isna(row[column]) else row[column], axis=1)

wdi_filtered.drop('mean_1980_2000', axis=1, inplace=True)

In [ ]:
wdi_filtered.sample(10)

In [ ]:
# Melt the dataset so that Country_Code and Series_Code remain cols,
#   while the years go from being columns to rows, with the Series_Code
#   value going into the Value col
melted_df = pd.melt(wdi_filtered, id_vars=['Country_Code', 'Series_Code'], 
                    value_vars=[str(year) for year in range(1980, 2021)],
                    var_name='Year', value_name='Value')

# Now pivot the dataset on the Series_Code column, so that each code
#   becomes its own column.  Convert the Year to float for easy joining
#   to the coral database dataframe
pivoted_df = melted_df.pivot_table(index=['Country_Code', 'Year'], 
                                   columns='Series_Code', 
                                   values='Value', 
                                   aggfunc='first').reset_index()
pivoted_df.Year = pivoted_df.Year.astype('float64')

# Merge the coral database and the pivoted wdi dataframe
gcb = pd.merge(gcb, pivoted_df, left_on=['Country_Code','Date_Year'], right_on=['Country_Code','Year'])

In [ ]:
wdi_filtered.sample(10)

In [ ]:
melted_df.sample(10)

In [ ]:
pivoted_df[pivoted_df.Country_Code == 'MDV'].sample(40)

In [ ]:
gcb.sample(10)

## Refine the Cols and Rows to keep

In [ ]:
# Drop columns that will not provide value to the analysis
gcb_drop_cols = [
    'Percent_Bleaching_Old_Method', 
    'bleach_intensity', 
    'Number_Bleached_Colonies', 
    'Percent_Hard_Coral', 
    'Percent_Macroalgae',
    'Sample_Method', 
    'Sample_Comments',
    'Site Comments',
    'Cover Comments', 
    'Site_Name'
    ]

gcb.drop(columns=gcb_drop_cols, axis=1, inplace=True)

In [ ]:
# Drop rows where Percent_Bleached_Value is not a number greater than or equal to 0
gcb = gcb[gcb.Percent_Bleached_Value >= 0]

# Drop rows where the SSTA records are null
gcb = gcb[gcb['SSTA'].notna()]

## Write Out Data

In [ ]:
# Write out parquet file, which is used in the analysis notebooks
# gcb.to_parquet("/work/data/Global_Coral_Bleaching_DB/gcb_v4.parquet")
# gcb.to_parquet("/datasets/s3/data/Global_Coral_Bleaching_DB/gcb_v4.parquet")

### Create Classification y-labels

In [ ]:
bins = [-1, 0, 10, 50, 100]
names = ['No Bleaching', 'Mild Bleaching', 'Moderate Bleaching', 'Severe Bleaching']

gcb['Bleached_Class'] = pd.cut(gcb['Percent_Bleached_Value'], bins, labels=names)


In [ ]:
# Write out parquet file, which is used in the analysis notebooks
gcb.to_parquet("/work/data/Global_Coral_Bleaching_DB/gcb_v5.parquet")
gcb.to_parquet("/datasets/s3/data/Global_Coral_Bleaching_DB/gcb_v5.parquet")